<h1>Тестовое задание.</h1>

<h3>Загрузка данных</h3>
<p>Скачаем данные, и обработаем их для ввода в базу данных.</p>

In [1]:
import pandas as pd

data = pd.read_csv('data_test.csv', sep=';')
data = data.rename(columns={'id события': 'id', 'ФИО участника события 1':'person_1', 'ФИО участника события 2':'person_2'})

In [2]:
person_1 = data.person_1
person_2 = data.person_2
#запишем всех участников из первого и второго столбца в одну переменную
persons = pd.Series(person_1.to_list() + person_2.to_list())

In [3]:
#проверим, число уникальных значений в переменной person и в person_1 и person_2 вместе 
persons.nunique()

9899

In [4]:
data.person_1.nunique() + data.person_2.nunique()

9924

Видим, что в переменной person меньше уникальных значений.  
Это значит, что есть участники, которые встречаются и в перовой и второй колонке.
<br><br>
Запишем всех участников в один файл <i>persons.csv</i>, чтобы далее загрузить их в базу данных.

In [5]:
pd.DataFrame(persons.unique(), columns=['person']).to_csv('persons.csv', encoding='utf-16', index=False)

<h2>Загрузим данные непосредственно в базу данных</h2>
<h3>Neo4j:</h3>

![Атрибуты узла](pics/neo_schema_second.jpg "Атрибуты узла")
![Атрибуты связи](pics/neo_schema_first.jpg "Атрибуты связи")

<h3>ArangoDB:</h3>

<p>Для данной базы данных создадим 2 коллекции Person и Event, в перовой будут содержаться вершины, а во второй грани.</p>
<p>Сначала загрузим ранее созданный файл persons.csv но конвертированный в формат json.</p>
<p>После успешной загрузки скачаем данные из полученной коллекции, чтобы преобразовать файл со связями(заменить имена на id, полученные автоматически при загрузки данных в коллекцию):</p>

In [6]:
import pandas as pd

persons = pd.read_json('results-People.json')

id = persons['_id']
Name = persons['person']

Name_key = dict(zip(Name, id))

data_test = pd.read_csv('data_test.csv', sep=';')
data_test = data_test.replace(Name_key).rename(columns={'id события': 'id_event', 
                                               'ФИО участника события 1': '_from',
                                               'ФИО участника события 2': '_to'})
data_test.to_csv('data_test_id.csv', index=False, encoding='utf-16')

<p>Теперь также конвертируем полученный файл в формат json и загрузим его в коллекцию Event.</p>
<p>Теперь создаём граф со следующими параметрами. После этого графовая база данных создана.</p>

![ArangoDB](pics/ArangoDB.jpg)

<pre>
Выведем 50 случайных вершин, чтобы посмотреть, как связаны данные

FOR person IN Person
    FOR v, e, p IN 1..5 ANY
    person Event
    LIMIT 50
    RETURN p   
        
</pre>
![Первый запрос Arango](pics/Arango_query(2).jpg)

<p>Скорее всего, большинство вершин имеют всего одну связь. Выведем вершины, которые имеют больше 1 грани.</p>
<pre>

FOR person IN Person
    FOR v, e, p IN 1..5 ANY
    person Event
    COLLECT per = v._id
    WITH COUNT INTO length
    FILTER length > 2
        FOR v, e, p IN 1..1 ANY
        per
        GRAPH Events  
        RETURN e  
           
</pre>
        
![Первый запрос Arango](pics/arango_query.jpg)

<h2>Поиск взаимосвязей(Neo4j)</h2>
<p>Посмотрим, как располагаются данные</p>
    
![Первый запрос](pics/query_first.jpg)

<p>Судя по всему, большинство узлов имеют всего одну связь. Также это было понятно по числу уникальных значений, что всего 101 раз какие-то узлы встречались повторно.</p>
<p>Посмотрим, есть ли узлы, имеющие больше двух связей:</p>

![Второй запрос](pics/query_second.jpg)

<p> Можем увидеть, что существует 6 связанных графов. Остальные узлы существуют только парами. </p>
<p> При чем эти 6 графов можно разделить на группы. У нас есть 3 графа которые имеют одну вершину, которая связанна со всеми остальными(графы а, б, д на картинке выше). Граф е состоит из 2 таких же графов, соединенных между собой выделенной на рисунке вершиной. Это первая группа графов. Вторая группа это граф в, который состоит из цепочки вершин, и граф г, который состоит из двух цепочек, которые соединены выделенным ребром.</p>
<p> Так как известно, что эти вершины обозначают людей, можно сделать вывод, что графы представляют собой социальные связи и существует 3 формы связей:</p>

* пары(просто 2 связанные вершины);
* графы с одной вершинной, связанной со всеми остальными(их можно назвать деревьями, если не учитывать направления дуг);
* цепочки.

<p> Посмотрим, много ли участников имеют больше двух связей и сколько связей они имеют всего:</p>

![Третий запрос](pics/query_third.jpg)

<p> Получим, что это всего 8 записей:</p>

![Третий запрос(таблица)](pics/query_third(table).jpg)


<p>Итак, можно сделать вывод, что данные графы, отражают взаимодействия между людьми, при чем у большинства людей имеется только одна связь. Тем не менее есть и люди, у которых присутствует большое количество связей, относительно остальных. Можно предположить, что эти люди выполняют какие-то особенные роли или являются организаторами бизнес-процесса, являющегося источником имеющихся данных.</p>

<p> Имея какие-либо ещё атрибуты у вершин, можно было бы сделать предположения о данных, по какому принципу объекты объединяются в группы или почему некоторые узлы имеют так много связей.</p>

<h2>Rest сервис для Neo4j:</h2>

In [7]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable


class App:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    def create_person(self):
        with self.driver.session(database="neo4j") as session:
            print("Введите ФИО участника полностью:")
            person_name = input()
            result = session.execute_write(
                self._create_person, person_name)
            for row in result:
                print("Создан участник: {p1}".format(p1=row['p1']))

    @staticmethod
    def _create_person(tx, person_name):
        query = (
            "CREATE (p1:participant { Name: $person_name }) "
            "RETURN p1"
        )
        result = tx.run(query, person_name=person_name)
        try:
            return [{"p1": row["p1"]["Name"]}
                    for row in result]
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

    def find_person(self):
        print("Введите ФИО участника полностью:")
        person_name = input()
        with self.driver.session() as session:
            result = session.execute_read(self._find_and_return_person, person_name)
            return result

    @staticmethod
    def _find_and_return_person(tx, person_name):
        query = (
            "MATCH (p:participant) "
            "WHERE p.Name = $person_name "
            "RETURN p, ID(p)"
        )
        result = tx.run(query, person_name=person_name)
        return [record.data() for record in result]


if __name__ == "__main__":
    uri = "neo4j+s://76720b56.databases.neo4j.io"
    user = "neo4j"
    password = "8wn7SBb35T68SeY9aiTC1W1Y-d6m1cTh2nVC_0h2D2Q"

In [8]:
app = App(uri, user, password)
app.create_person()
app.close()

Введите ФИО участника полностью:
Мамонов Александр Александрович
Создан участник: Мамонов Александр Александрович


In [9]:
app = App(uri, user, password)
result = app.find_person()
print(result)
app.close()

Введите ФИО участника полностью:
Мамонов Александр Александрович
[{'p': {'Name': 'Мамонов Александр Александрович'}, 'ID(p)': 596}]


<h2>Rest сервис для ArangoDB:</h2>

In [10]:
from pyArango.connection import *

def create_person(connection):
    print("Введите ФИО участника полностью:")
    person_name = input()
    query = ( 
        "INSERT {"
        "person: '" + person_name +
        "'} INTO Person"
    )
    Database(connection, 'People').AQLQuery(query)
    print("Создан участник: {p1}".format(p1=person_name))
    
def find_person(connection):
    print("Введите ФИО участника полностью:")
    person_name = input()
    query = ( 
        "FOR per IN Person " 
        "FILTER per.person == '"+ person_name + "' "
        "RETURN per"
    )
    result = Database(connection, 'People').AQLQuery(query)
    return result

if __name__=="__main__":
    url = "https://26e19b52c692.arangodb.cloud:8529/"
    user = "root"
    password = "DN9HftSvcy4XExANCBG5"
    conn = Connection(arangoURL=url, username=user, password=password)
    conn.resetSession(user, password)

In [11]:
conn = Connection(arangoURL=url, username=user, password=password)
create_person(conn)
conn.resetSession(user, password)

Введите ФИО участника полностью:
Мамонов Александр Александрович
Создан участник: Мамонов Александр Александрович


In [12]:
conn = Connection(arangoURL=url, username=user, password=password)
result = find_person(conn)
print(result)
conn.resetSession(user, password)

Введите ФИО участника полностью:
Мамонов Александр Александрович
[{'_key': '93234', '_id': 'Person/93234', '_rev': '_foyahDe--_', 'person': 'Мамонов Александр Александрович'}]
